# Weight pruning demo
<h3> Weight pruning demo on image classification model </h3>

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import glob
import pandas as pd
import shutil 
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization


In [16]:
#Data preparation
train_file = glob.glob('train/**/*.jpg')
for i in train_file:
    fileName = os.path.basename(i)
    if 'dog' in i:
        shutil.move(i,'train/dog/'+fileName)
    else:
        shutil.move(i,'train/cat/'+fileName)
        

In [8]:
train_dir = 'train/'
batch_size = 32
img_height = 128
img_width = 128        

In [9]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.25)
  ]
)


In [11]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
label_mode='binary')

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
label_mode='binary')

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [13]:
def getModel(input_shape=(img_height,img_width,3)):
    
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=input_shape))    
    model.add(data_augmentation)
    model.add(layers.experimental.preprocessing.Rescaling(1./255))

    model.add(Conv2D(32, (3, 3), activation=layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation=layers.LeakyReLU()))
    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation=layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation=layers.LeakyReLU()))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    
    return model
    #model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [14]:
model = getModel()

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 64)       

In [15]:

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
save_model_path = 'weights.hdf5'
cp = keras.callbacks.ModelCheckpoint(filepath=save_model_path, monitor='val_loss', save_best_only=True, verbose=1)
earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit_generator(
        train_ds,
        steps_per_epoch=len(train_ds),
        epochs=10,
        validation_data=val_ds,
        validation_steps=len(val_ds),
callbacks=[cp,lr_reducer,earlystopper])


Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 0.6920 - accuracy: 0.6449
Epoch 00001: val_loss improved from inf to 0.58765, saving model to weights.hdf5
625/625 [==============================] - 214s 342ms/step - loss: 0.6920 - accuracy: 0.6449 - val_loss: 0.5877 - val_accuracy: 0.6824
Epoch 2/10
625/625 [==============================] - ETA: 0s - loss: 0.5649 - accuracy: 0.7078
Epoch 00002: val_loss improved from 0.58765 to 0.49677, saving model to weights.hdf5
625/625 [==============================] - 197s 315ms/step - loss: 0.5649 - accuracy: 0.7078 - val_loss: 0.4968 - val_accuracy: 0.7572
Epoch 3/10
623/625 [============================>.] - ETA: 0s - loss: 0.5286 - accuracy: 0.7378
Epoch 00003: val_loss improved from 0.49677 to 0.46054, saving model to weights.hdf5
625/625 [==============================] - 138s 220ms/step - loss: 0.5286 - accuracy: 0.7379 - val_loss: 0.4605 - val_accuracy: 0.7850
Epoch 4/10
624/625 [============================>.] - ET

In [46]:
base_loss, base_acc = model.evaluate(val_ds, verbose=0)
print("Validation accuracy: ", base_acc)
print("Validation loss: ", base_loss)

Validation accuracy:  0.840399980545044
Validation loss:  0.36342158913612366


<h2><b>Weight pruning of above model<b></h2>
    
#https://blog.tensorflow.org/2019/05/tf-model-optimization-toolkit-pruning-API.html</n>

#https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide
    
<h5>Weight pruning means eliminating unnecessary values in the weight tensors. We are practically setting the neural network parameters’ values to zero to remove what we estimate are unnecessary connections between the layers of a neural network. This is done during the training process to allow the neural network to adapt to the changes.</h5>
<img src='misc/neuralnetworklayersbeforeandafterpruning.png' width="500" height="600" />

In [17]:
import tensorflow_model_optimization as tfmot

base_model =getModel()
base_model.load_weights('weights.hdf5')
# whole model is pruned
def apply_pruning(layer):
  if(
      isinstance(layer, tf.keras.Sequential) 
      or isinstance(layer, layers.experimental.preprocessing.Rescaling)
  ):
    return layer
  print("pruned layer: ",layer)
  return tfmot.sparsity.keras.prune_low_magnitude(layer)
  

model_for_pruning = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_pruning,
)
model_for_pruning.summary()

pruned layer:  <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f95cc225320>
pruned layer:  <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f95cc21d198>
pruned layer:  <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f95cc21d6a0>
pruned layer:  <tensorflow.python.keras.layers.core.Dropout object at 0x7f95df7aa710>
pruned layer:  <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f95747f0978>
pruned layer:  <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f95c006acf8>
pruned layer:  <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f95c006ab00>
pruned layer:  <tensorflow.python.keras.layers.core.Dropout object at 0x7f95746f83c8>
pruned layer:  <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f95cc574be0>
pruned layer:  <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f95cc55c780>
pruned layer:  <tensorflow.pyth

# 1
<h3>Non Trainable params is 12,940,085 </h3>
<h2>All layers are pruned<h2>

In [18]:
import tempfile
log_dir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
]

model_for_pruning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model_for_pruning.fit_generator(
        train_ds,
        steps_per_epoch=len(train_ds),
        epochs=2,
        validation_data=val_ds,
        validation_steps=len(val_ds),
callbacks=callbacks)



Epoch 1/2
  1/625 [..............................] - ETA: 0s - loss: 0.2457 - accuracy: 0.9062WARNING:tensorflow:From /home/contentcreation/anaconda3/envs/tensorflow-2/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
625/625 [==============================] - 141s 225ms/step - loss: 0.3991 - accuracy: 0.8181 - val_loss: 0.4186 - val_accuracy: 0.8082
Epoch 2/2
625/625 [==============================] - 139s 223ms/step - loss: 0.3891 - accuracy: 0.8251 - val_loss: 0.3750 - val_accuracy: 0.8358


In [48]:
loss1, acc1 = model_for_pruning.evaluate(val_ds, verbose=0)
print("Validation accuracy: ", acc1)
print("Validation loss: ", loss1)

Validation accuracy:  0.8357999920845032
Validation loss:  0.3749958276748657


In [25]:
model_for_pruning.save_weights("model_all_pruning.h5")

In [22]:
base_model_sel =getModel()
base_model_sel.load_weights('weights.hdf5')
def apply_pruning_to_dense(layer):
  if  isinstance(layer, tf.keras.layers.Dense):
    print(layer)
    return tfmot.sparsity.keras.prune_low_magnitude(layer)
  return layer
  

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense` 
# to the layers of the model.
model_for_pruning_selective = tf.keras.models.clone_model(
    base_model_sel,
    clone_function=apply_pruning_to_dense,
)
model_for_pruning_selective.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_6 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_24 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 61, 61, 64)       

# 2
<h3>Non Trainable params is 12,847,044  </h3>
<h2>Pruning only dense layers <h2>

In [23]:
model_for_pruning_selective.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model_for_pruning_selective.fit_generator(
        train_ds,
        steps_per_epoch=len(train_ds),
        epochs=2,
        validation_data=val_ds,
        validation_steps=len(val_ds),
callbacks=callbacks)
model_for_pruning_selective.save_weights("model_for_pruning_selective.h5")

Epoch 1/2
625/625 [==============================] - 137s 219ms/step - loss: 0.4171 - accuracy: 0.8088 - val_loss: 0.5714 - val_accuracy: 0.7610
Epoch 2/2
625/625 [==============================] - 137s 219ms/step - loss: 0.4040 - accuracy: 0.8139 - val_loss: 0.4078 - val_accuracy: 0.8250


In [43]:
loss1, acc1 = model_for_pruning_selective.evaluate(val_ds, verbose=0)
print("Validation accuracy: ", acc1)
print("Validation loss: ", loss1)

Validation accuracy:  0.824999988079071
Validation loss:  0.4077540934085846


# Model Comparision

In [55]:
nonPrune = os.path.getsize("weights.hdf5")
prune = os.path.getsize("model_all_pruning.h5")
print("Non pruned accuracy: ", base_acc ," VS pruned accuracy: ", acc1)

print("Model size saving in pruned model {} %".format( int((1 - (prune/nonPrune))*100 )))
print("Difference in accuracy of pruned model {}".format(acc1-base_acc))


Non pruned accuracy:  0.840399980545044  VS pruned accuracy:  0.8357999920845032
Model size saving in pruned model 33 %
Difference in accuracy of pruned model 0.0045999884605407715


# Model size saving in pruned model 33 %
# Difference in accuracy of pruned model 0.0045999884605407715

<h3>An immediate benefit from this work is disk compression: sparse tensors are amenable to compression.  we can reduce the size of the model for its storage and/or transmission </h3>